Посмотрим процедуру создания Dataset в торче на очень простом примере. 

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

Датасет должен уметь возвращать свою длину и пары фичи - ytrue. Поэтому перегрузим три метода:

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, txt, labels):
        self.labels = labels
        self.text = text
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        sample = {"Text": text, "Class": label}
        return sample

In [ ]:
# сделаем игрушечный датасетик
text = ['Happy', 'Amazing', 'Sad', 'Unhappy', 'Glum']
labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']
# соберем из него датафрейм
text_labels_df = pd.DataFrame({'Text': text, 'Labels': labels})
# и передадим в класс Dataset
TD = CustomTextDataset(text_labels_df['Text'], text_labels_df['Labels'])

In [ ]:
Посмотрим, как он будет работать
print('\nFirst iteration of data set: ', next(iter(TD)), '\n')
print('Length of data set: ', len(TD), '\n')
print('Entire data set: ', list(DataLoader(TD)), '\n')

Иногда нам еще бывает нужна специальная функция для предобработки батча перед тем, как его передавать в модель. Ее можно имплементировать в классе датасета или просто написать отдельно. Она передается в параметрах DataLoader. Мы напишем тоже игрушечную функцию, которая ничего особенного делать не будет, сымитирует превращение текстов в тензора. 

In [ ]:
def collate_batch(batch):    
    word_tensor = torch.tensor([[1.], [0.], [45.]])
    label_tensor = torch.tensor([[1.]])
    
    text_list, classes = [], []    
    for (_text, _class) in batch:
        text_list.append(word_tensor)
        classes.append(label_tensor)     
        text = torch.cat(text_list)
        classes = torch.tensor(classes)     
        return text, classes

Посмотрим, как это будет работать:

In [ ]:
DL_DS = DataLoader(TD, batch_size=2, shuffle=True)
for (idx, batch) in enumerate(DL_DS):    
    print(idx, 'Text data: ', batch['Text'])    
    print(idx, 'Class data: ', batch['Class'], '\n')

In [ ]:
DL_DS = DataLoader(TD, batch_size=2, collate_fn=collate_batch)
for (idx, batch) in enumerate(DL_DS): 
    print(f'{idx}.\nFeatures: {batch[0]}\nY: {batch[1]}')

Посмотрим функцию collate_fn подробнее. 

collate_fn получает список кортежей (если \_\_getitem\_\_ в датасете их возвращает) или просто обычный список чего угодно. Основная задача этой функции - собрать батч, не тратя время на соображения, как там наши данные поделить и еще пошаффлить. У DataLoader есть встроенная функция collate_fn, которая используется, если мы не передали кастомной. 

Допустим, у нас есть другой игрушечный датасет:

In [ ]:
data = np.array([
    [0.1, 7.4, 0],
    [-0.2, 5.3, 0],
    [0.2, 8.2, 1],
    [0.2, 7.7, 1]])
print(data)

На самом деле мы можем его прямиком сунуть в DataLoader, он сам разберется:

In [ ]:
loader = DataLoader(data, batch_size=2, shuffle=False)
batch = next(iter(loader))
print(batch)

DataLoader не нашел игреки, правда, но догадался разделить по объектам. Умеет он это делать и со словарями:

In [ ]:
from pprint import pprint
# теперь датасет - список словарей
dict_data = [
    {'x1': 0.1, 'x2': 7.4, 'y': 0},
    {'x1': -0.2, 'x2': 5.3, 'y': 0},
    {'x1': 0.2, 'x2': 8.2, 'y': 1},
    {'x1': 0.2, 'x2': 7.7, 'y': 10},
]
pprint(dict_data)

In [ ]:
loader = DataLoader(dict_data, batch_size=2, shuffle=False)
batch = next(iter(loader))
pprint(batch)

Проблемки у дефолтного collate_fn обычно начинаются, если наши данные - разных размеров. Когда это может быть? Когда мы работаем с текстами: например, каждый объект - это предложение, а все предложения, как мы знаем, разной длины. Торч с батчами разной длины работать не умеет. Такой код вызовет ошибку:

In [ ]:
nlp_data = [
    {'tokenized_input': [1, 4, 5, 9, 3, 2],
     'label':0},
    {'tokenized_input': [1, 7, 3, 14, 48, 7, 23, 154, 2],
     'label':0},
    {'tokenized_input': [1, 30, 67, 117, 21, 15, 2],
     'label':1},
    {'tokenized_input': [1, 17, 2],
     'label':0},
]
loader = DataLoader(nlp_data, batch_size=2, shuffle=False)
batch = next(iter(loader))

Что делать? Приходится добивать предложения т.н. паддингами (говорят - паддить). Тут есть несколько стратегий, одна из которых предполагает, что мы берем несколько предложений в батч, выбираем самое длинное из них, а остальные добиваем специальными штуками до длины самого длинного; другая предполагает, что мы выбираем среднюю длину по батчу и обрезаем слишком длинные, а слишком короткие добиваем. Причем можно либо весь датасет западдить по самому длинному предложению, либо паддить каждый батч на лету (обычно делается последнее). 

In [ ]:
from torch.nn.utils.rnn import pad_sequence # торч будет все делать за нас

def custom_collate(data): 
    # отделим игреки от фич
    inputs = [torch.tensor(d['tokenized_input']) for d in data] 
    labels = [d['label'] for d in data] 

    inputs = pad_sequence(inputs, batch_first=True) 
    labels = torch.tensor(labels) 

    return { 
        'tokenized_input': inputs,
        'label': labels
    }

loader = DataLoader(nlp_data, batch_size=2, shuffle=False, collate_fn=custom_collate) 

iter_loader = iter(loader)
batch1 = next(iter_loader)
pprint(batch1)
batch2 = next(iter_loader)
pprint(batch2)

Что же это за batch_first такой? Дело в том, как работает pad_sequence. Она принимает на вход объект размеров $L \times *$ (L - длина списка, * - остальные размерности) и возвращает новый объект размерности $T \times B \times *$, где Т - длина самой длинной последовательности, B - длина батча. То есть:

In [ ]:
a = torch.ones(8, 1) # условное предложение из 8 слов
print('Так выглядит самая длинная последовательность:', a)
b = torch.ones(5, 1)
c = torch.ones(3, 1)
batch = [a, b, c]
print(f'Длина нашего батча: {len(batch)}')
print(pad_sequence(batch).size())
print(pad_sequence(batch))

Получается, pad_sequence все напутал и мы вместо батча из трех предложений получаем неведомо что из 8 неведомо чего. Параметр batch_first приходит на помощь и, собственно, говорит pad_sequence, что нужно поменять размерность местами и первым возвращать размерность батча. 

In [ ]:
a = torch.ones(8, 1) # условное предложение из 8 слов
print('Так выглядит самая длинная последовательность:', a)
b = torch.ones(5, 1)
c = torch.ones(3, 1)
batch = [a, b, c]
print(f'Длина нашего батча: {len(batch)}')
print(pad_sequence(batch, batch_first=True).size())
print(pad_sequence(batch, batch_first=True))